In [ ]:
import json

# Struktur data pohon keputusan
class NodePohon:
    def __init__(self, fitur=None, cabang_ya=None, cabang_tidak=None, nilai=None):
        self.fitur = fitur
        self.cabang_ya = cabang_ya
        self.cabang_tidak = cabang_tidak
        self.nilai = nilai

    def to_dict(self):
        if self.nilai is not None:
            return {"nilai": self.nilai}
        return {
            "fitur": self.fitur,
            "cabang_ya": self.cabang_ya.to_dict(),
            "cabang_tidak": self.cabang_tidak.to_dict()
        }

    @staticmethod
    def from_dict(data):
        if "nilai" in data:
            return NodePohon(nilai=data["nilai"])
        return NodePohon(
            fitur=data["fitur"],
            cabang_ya=NodePohon.from_dict(data["cabang_ya"]),
            cabang_tidak=NodePohon.from_dict(data["cabang_tidak"])
        )

# Fungsi prediksi
def prediksi(pohon, jawaban_user):
    if pohon.nilai is not None:
        return pohon.nilai
    nilai_fitur = jawaban_user[pohon.fitur]
    if nilai_fitur == 1:
        return prediksi(pohon.cabang_ya, jawaban_user)
    else:
        return prediksi(pohon.cabang_tidak, jawaban_user)

# Pertanyaan per dimensi
pertanyaan_per_dimensi = {
    "Introvert": [
        "Saya lebih nyaman menyendiri daripada berada di keramaian.",
        "Saya merasa lelah setelah banyak bersosialisasi.",
        "Saya suka merenung dan berpikir sendiri.",
        "Saya sering menghindari pembicaraan basa-basi.",
        "Saya lebih memilih komunikasi lewat tulisan daripada bicara langsung."
    ],
    "Sensing": [
        "Saya lebih fokus pada fakta daripada teori.",
        "Saya memperhatikan detail kecil dalam pekerjaan saya.",
        "Saya percaya pada pengalaman langsung daripada intuisi.",
        "Saya lebih suka instruksi yang jelas dan praktis.",
        "Saya mengandalkan panca indera untuk memahami dunia."
    ],
    "Thinking": [
        "Saya lebih memprioritaskan logika daripada perasaan dalam mengambil keputusan.",
        "Saya merasa nyaman memberi kritik jika diperlukan.",
        "Saya lebih percaya pada keadilan daripada belas kasihan.",
        "Saya memilih keefisienan daripada keharmonisan.",
        "Saya memisahkan emosi dari analisis saat bekerja."
    ],
    "Judging": [
        "Saya suka merencanakan sesuatu jauh-jauh hari.",
        "Saya merasa tidak nyaman jika jadwal berubah.",
        "Saya suka membuat daftar tugas (to-do list) dan mengikutinya.",
        "Saya merasa puas setelah menyelesaikan pekerjaan tepat waktu.",
        "Saya suka semua hal berjalan sesuai rencana."
    ]
}

# Penjelasan hasil MBTI
penjelasan_mbti = {
    "ISTJ": "ISTJ (Logistician): Tipe ini dikenal sangat bertanggung jawab, terorganisir, dan berorientasi pada fakta. Cocok untuk pekerjaan yang memerlukan struktur dan perencanaan.",
    "ISFJ": "ISFJ (Defender): Tipe ini sangat peduli, setia, dan suka membantu. Mereka cenderung diam namun perhatian pada kebutuhan orang lain.",
    "INTJ": "INTJ (Architect): Tipe ini strategis, logis, dan suka merancang masa depan. Mereka lebih nyaman dengan teori dan konsep besar.",
    "INFJ": "INFJ (Advocate): Tipe ini idealis, visioner, dan sangat peduli dengan nilai-nilai kemanusiaan. Mereka memiliki intuisi yang tajam.",
    "ESTJ": "ESTJ (Executive): Tipe ini pemimpin alami, suka mengambil alih tanggung jawab, dan mengatur segala sesuatu dengan efisien.",
    "ESFJ": "ESFJ (Consul): Mereka ramah, suka bersosialisasi, dan peduli terhadap kesejahteraan kelompok. Sangat cocok jadi pendukung sosial.",
    "ENTP": "ENTP (Debater): Mereka inovatif, suka tantangan, dan sangat komunikatif. Senang mencoba hal-hal baru dan memecahkan masalah secara kreatif.",
    "ENFP": "ENFP (Campaigner): Energik, penuh semangat, dan sangat ekspresif. Mereka suka menjalin hubungan dan menginspirasi orang lain."
}

# Struktur pohon keputusan 1 blok
root = NodePohon(
    fitur="Introvert",
    cabang_ya=NodePohon(
        fitur="Sensing",
        cabang_ya=NodePohon(
            fitur="Thinking",
            cabang_ya=NodePohon(nilai="ISTJ"),
            cabang_tidak=NodePohon(nilai="ISFJ")
        ),
        cabang_tidak=NodePohon(
            fitur="Judging",
            cabang_ya=NodePohon(nilai="INTJ"),
            cabang_tidak=NodePohon(nilai="ENTP")
        )
    ),
    cabang_tidak=NodePohon(
        fitur="Sensing",
        cabang_ya=NodePohon(
            fitur="Thinking",
            cabang_ya=NodePohon(nilai="ESTJ"),
            cabang_tidak=NodePohon(nilai="ESFJ")
        ),
        cabang_tidak=NodePohon(
            fitur="Judging",
            cabang_ya=NodePohon(nilai="INFJ"),
            cabang_tidak=NodePohon(nilai="ENFP")
        )
    )
)

# Fungsi utama
def main():
    jawaban_dimensi = {}
    print("Jawablah pertanyaan berikut dengan 0 (tidak setuju) atau 1 (setuju):\n")
    for dimensi, daftar_pertanyaan in pertanyaan_per_dimensi.items():
        skor = 0
        for pertanyaan in daftar_pertanyaan:
            while True:
                try:
                    jawab = int(input(f"{pertanyaan} (0/1): "))
                    if jawab in [0, 1]:
                        skor += jawab
                        break
                    else:
                        print("Masukkan hanya angka 0 atau 1.")
                except ValueError:
                    print("Masukkan angka 0 atau 1.")
        jawaban_dimensi[dimensi] = 1 if skor >= 3 else 0

    hasil = prediksi(root, jawaban_dimensi)
    print("\nHasil Prediksi Tipe MBTI Kamu:", hasil)

    # Penjelasan MBTI
    print("\nPenjelasan Tipe Kamu:")
    print(penjelasan_mbti.get(hasil, "Penjelasan tidak tersedia."))

    # Simpan struktur pohon ke file JSON (opsional)
    with open("pohon_mbti.json", "w") as f:
        json.dump(root.to_dict(), f, indent=2)
    print("\nPohon disimpan sebagai 'pohon_mbti.json'")

# Jalankan
if __name__ == "__main__":
    main()


Jawablah pertanyaan berikut dengan 0 (tidak setuju) atau 1 (setuju):


Hasil Prediksi Tipe MBTI Kamu: ISTJ

Pohon disimpan sebagai pohon_mbti.json
